# **OpenAI Agents SDK Baseline Email Assistant**

# Baseline Email Assistant

![architecture_pic](https://github.com/panaversity/learn-agentic-ai/blob/main/01_openai_agents/16_memory/01_langmem/00_baseline_email_assistant/img/memory_course_email.png?raw=true)

This lesson builds an email assistant that:
- Uses OpenAI Agents SDK
- Classifies incoming messages (respond, ignore, notify)
- Drafts responses
- Schedules meetings

We'll start with a simple implementation - one that uses hard-coded rules to handle emails.

# Install openai-agents SDK

In [19]:
!pip install -Uq openai-agents pydantic langmem

# Make your Notebook capable of running asynchronous functions.
Both Jupyter notebooks and Python’s asyncio library utilize event loops, but they serve different purposes and can sometimes interfere with each other.

The nest_asyncio library allows the existing event loop to accept nested event loops, enabling asyncio code to run within environments that already have an event loop, such as Jupyter notebooks.

In summary, both Jupyter notebooks and Python’s asyncio library utilize event loops to manage asynchronous operations. When working within Jupyter notebooks, it’s essential to be aware of the existing event loop to effectively run asyncio code without conflicts.

In [20]:
import nest_asyncio
nest_asyncio.apply()

In [36]:
from pydantic import BaseModel, Field
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
    output_guardrail,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)
from google.colab import userdata
from typing import Dict, Any
from typing_extensions import TypedDict, Literal, Annotated


In [49]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

In [61]:

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)


## **Setup a Profile, Prompt Instructions and Example Email**

In [62]:
profile = {
    "name": "John",
    "full_name": "John Doe",
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

In [63]:
prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage John's tasks efficiently."
}

In [64]:
# Example incoming email
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

class Email(BaseModel):
    from_: str = Field(alias="from")
    to: str
    subject: str
    body: str

email_model = Email(**email)
print(email_model.model_dump_json(by_alias=True))

{"from":"Alice Smith <alice.smith@company.com>","to":"John Doe <john.doe@company.com>","subject":"Quick question about API documentation","body":"\nHi John,\n\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\n\nThanks!\nAlice"}


In [65]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

## **Prompt Templates**

In [66]:
# Triage system prompt template
triage_system_prompt_template = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Background >
{user_profile_background}.
</ Background >

< Instructions >

{name} gets lots of emails. Your job is to categorize each email into one of three categories:

1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that {name} should know about but doesn't require a response
3. RESPOND - Emails that need a direct response from {name}

Classify the below email into one of these categories.

</ Instructions >

< Rules >
Emails that are not worth responding to:
{triage_no}

There are also other things that {name} should know about, but don't require an email response. For these, you should notify {name} (using the `notify` response). Examples of this include:
{triage_notify}

Emails that are worth responding to:
{triage_email}
</ Rules >

< Few shot examples >
{examples}
</ Few shot examples >
"""





# Triage User Prompt Template
triage_user_prompt_template = """
Please determine how to handle the below email thread:

From: {author}
To: {to}
Subject: {subject}
{email_thread}"""





## **A function to create a prompt using f-string**

In [67]:
def create_prompt(template: str, variables: Dict[str, any]) -> str:
    """Creates a prompt using an f-string and a dictionary of variables."""
    try:
        return template.format(**variables)
    except KeyError as e:
        return f"Error: Missing variable '{e.args[0]}' in the provided dictionary."

In [68]:
system_prompt = create_prompt(triage_system_prompt_template, {
    "full_name": profile["full_name"],
    "name":profile["name"],
    "examples": None,
    "user_profile_background": profile["user_profile_background"],
    "triage_no" : prompt_instructions["triage_rules"]["ignore"],
    "triage_notify": prompt_instructions["triage_rules"]["notify"],
    "triage_email" : prompt_instructions["triage_rules"]["respond"],
  }
)

In [69]:
user_prompt = create_prompt(triage_user_prompt_template, {
    "author": email["from"],
    "to": email["to"],
    "subject": email["subject"],
    "email_thread" : email["body"],
  }
)

## **Triage Agent**

In [74]:
triage_agent = Agent(
    name="Triage Agent",
    instructions=system_prompt,
    output_type=Router
)

In [75]:
triage_result = await Runner.run(triage_agent, user_prompt, run_config = config)
print(triage_result.final_output.classification)
print(triage_result.final_output.reasoning)


BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'Invalid JSON payload received. Unknown name "metadata": Cannot find field.\nInvalid JSON payload received. Unknown name "store": Cannot find field.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'description': 'Invalid JSON payload received. Unknown name "metadata": Cannot find field.'}, {'description': 'Invalid JSON payload received. Unknown name "store": Cannot find field.'}]}]}}]

## **Response Agent, Define Tools**

In [76]:
from agents import function_tool

In [77]:
@function_tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'"

In [78]:
@function_tool
def schedule_meeting(
    attendees: list[str],
    subject: str,
    duration_minutes: int,
    preferred_day: str
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees"



In [79]:
@function_tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

## **Response Agent, Define Prompt**

In [80]:
response_prompt_template = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

In [81]:
response_system_prompt = create_prompt(response_prompt_template, {
    "full_name": profile["full_name"],
    "name":profile["name"],
    "instructions": prompt_instructions["agent_instructions"],
  }
)
print(response_system_prompt)


< Role >
You are John Doe's executive assistant. You are a top-notch executive assistant who cares about John performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage John's communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
</ Tools >

< Instructions >
Use these tools when appropriate to help manage John's tasks efficiently.
</ Instructions >



In [82]:
response_agent = Agent(
    name="Response agent",
    instructions=response_system_prompt,
    tools=[write_email, schedule_meeting, check_calendar_availability]
)


In [84]:
response_result = await Runner.run(response_agent, "what is my availability for tuesday?", run_config = config)
print(response_result.final_output)


BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'Invalid JSON payload received. Unknown name "metadata": Cannot find field.\nInvalid JSON payload received. Unknown name "store": Cannot find field.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'description': 'Invalid JSON payload received. Unknown name "metadata": Cannot find field.'}, {'description': 'Invalid JSON payload received. Unknown name "store": Cannot find field.'}]}]}}]

## **Create the Overall Flow in Python**

In [85]:
async def triage_router(email: Email):

  user_prompt = create_prompt(triage_user_prompt_template, {
    "author": email.from_,
    "to": email.to,
    "subject": email.subject,
    "email_thread" : email.body,
  })

  # print(user_prompt)

  triage_result = await Runner.run(triage_agent, user_prompt, run_config = config)
  print(triage_result.final_output)
  print("Triage History: ", triage_result.to_input_list())

  if triage_result.final_output.classification == "respond":
        print("📧 Classification: RESPOND - This email requires a response")
        response_result = await Runner.run(response_agent, f"Respond to the email {email.model_dump_json(by_alias=True)}", run_config = config)
        print(response_result.final_output)
        print("Response History", response_result.to_input_list())
  elif triage_result.final_output.classification == "ignore":
      print("🚫 Classification: IGNORE - This email can be safely ignored")
  elif triage_result.final_output.classification == "notify":
      # If real life, this would do something else
      print("🔔 Classification: NOTIFY - This email contains important information")
  else:
      raise ValueError(f"Invalid classification: {triage_result.final_output.classification}")


## **Now Test the Triage and Response Agents Working Together**

In [86]:
email_input = {
    "from": "Marketing Team <marketing@amazingdeals.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "🔥 EXCLUSIVE OFFER: Limited Time Discount on Developer Tools! 🔥",
    "body": """Dear Valued Developer,

Don't miss out on this INCREDIBLE opportunity!

🚀 For a LIMITED TIME ONLY, get 80% OFF on our Premium Developer Suite!

✨ FEATURES:
- Revolutionary AI-powered code completion
- Cloud-based development environment
- 24/7 customer support
- And much more!

💰 Regular Price: $999/month
🎉 YOUR SPECIAL PRICE: Just $199/month!

🕒 Hurry! This offer expires in:
24 HOURS ONLY!

Click here to claim your discount: https://amazingdeals.com/special-offer

Best regards,
Marketing Team
---
To unsubscribe, click here
""",
}

typed_email = Email(**email_input)
print(typed_email.model_dump_json(by_alias=True))

{"from":"Marketing Team <marketing@amazingdeals.com>","to":"John Doe <john.doe@company.com>","subject":"🔥 EXCLUSIVE OFFER: Limited Time Discount on Developer Tools! 🔥","body":"Dear Valued Developer,\n\nDon't miss out on this INCREDIBLE opportunity!\n\n🚀 For a LIMITED TIME ONLY, get 80% OFF on our Premium Developer Suite!\n\n✨ FEATURES:\n- Revolutionary AI-powered code completion\n- Cloud-based development environment\n- 24/7 customer support\n- And much more!\n\n💰 Regular Price: $999/month\n🎉 YOUR SPECIAL PRICE: Just $199/month!\n\n🕒 Hurry! This offer expires in:\n24 HOURS ONLY!\n\nClick here to claim your discount: https://amazingdeals.com/special-offer\n\nBest regards,\nMarketing Team\n---\nTo unsubscribe, click here\n"}


In [ ]:
await triage_router(typed_email)

In [87]:
email_input2 = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

typed_email2 = Email(**email_input2)

In [88]:
await triage_router(typed_email2)

BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'Invalid JSON payload received. Unknown name "metadata": Cannot find field.\nInvalid JSON payload received. Unknown name "store": Cannot find field.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'description': 'Invalid JSON payload received. Unknown name "metadata": Cannot find field.'}, {'description': 'Invalid JSON payload received. Unknown name "store": Cannot find field.'}]}]}}]